In [1]:
import pandas as pd
import xmlrpc.client
import re


In [2]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id','name'#'date_start'
                , 'product_id', 'product_qty','valor_unitario_mrp','date_finished']}
)


ops = pd.DataFrame(ordenes_produccion) # todas las OP cerradas

ops["date_finished"] = pd.to_datetime(ops["date_finished"])



# 1) Quedarnos con el nombre del producto dentro de product_id
#    (Odoo devuelve [id, "nombre"])
ops['product_name_raw'] = ops['product_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

# 2) Quitar el código entre corchetes "[...]" + espacios iniciales
# 3) Quitar todo lo que esté dentro de paréntesis "(...)" + espacios
ops['producto'] = (
    ops['product_name_raw']
      .astype(str)
      .str.replace(r'^\s*\[.*?\]\s*', '', regex=True)   # elimina "[CODE] "
      .str.replace(r'\(.*?\)', '', regex=True)          # elimina "(...)"
      .str.strip()
)

# (opcional) normalizar espacios internos
ops['producto'] = ops['producto'].str.replace(r'\s+', ' ', regex=True)

# Tu regex original (funciona para códigos alfanuméricos)
ops['product_ref'] = ops["product_name_raw"].str.extract(r"\[([A-Z0-9]+)\]")

# Extraer carcasas (captura lo que tenga letras, números y espacios dentro de corchetes)
ops['carcasa_ref'] = ops["product_name_raw"].str.extract(r"\[(CARCASA.*?)\]")

# Combinar: si no hay product_ref, usar carcasa_ref
ops['product_ref'] = ops['product_ref'].fillna(ops['carcasa_ref'])

ops["nom_produccion"] = ops["producto"].str.extract(r"^(\S+)")


# Eliminar la columna auxiliar
ops = ops.drop(columns=['carcasa_ref'])


prototypes = ops[ops["product_qty"]<=2]

ops = ops[ops["product_qty"]>2] #Esta regla elimina todos los prototipos o op's donde se fabrica una unica unidad




tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)

pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")


pareto_merge = pareto_prod[["nom_produccion","Clasificacion"]]

#Demora 1 minuto

In [3]:
test = test[["nom_produccion","display_name","total_nbr_minimo"]]
test_clss = pareto_merge.merge(test , on ="nom_produccion" ,how="right")

In [4]:
test_nan = test_clss[test_clss["Clasificacion"].isna()].copy()
test_all = test_clss[test_clss["Clasificacion"].notna()].copy()

In [5]:
test_def =test_all.groupby(["nom_produccion","Clasificacion"]).agg({"total_nbr_minimo":"sum"}).reset_index()
test_def.sample(10)

,nom_produccion,Clasificacion,total_nbr_minimo
256,DA4023,B,0.0
378,DA4946,A,0.0
95,DA2520,C,0.0
605,DA9151,B,0.0
422,DA8012,B,0.0
180,DA2881UHE,C,0.0
311,DA4369,B,0.0
225,DA3068,C,0.0
486,DA8121,C,0.0
169,DA2855,A,0.0


In [6]:
print(f"Primera fecha encontrada",ops["date_finished"].min())
print(f"Ultima fecha encontrada",ops["date_finished"].max())

print("_________________________________________________________")
print("")
ops.info()

Primera fecha encontrada 2016-05-05 12:56:41
Ultima fecha encontrada 2025-11-06 08:14:08
_________________________________________________________

<class 'pandas.core.frame.DataFrame'>
Index: 12164 entries, 0 to 12257
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   valor_unitario_mrp  12164 non-null  float64       
 1   product_id          12164 non-null  object        
 2   date_finished       12164 non-null  datetime64[ns]
 3   product_qty         12164 non-null  float64       
 4   id                  12164 non-null  int64         
 5   name                12164 non-null  object        
 6   product_name_raw    12164 non-null  object        
 7   producto            12164 non-null  object        
 8   product_ref         12164 non-null  object        
 9   nom_produccion      12164 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 1.0

In [7]:
ops_def = ops[["name","product_ref","nom_produccion","producto","valor_unitario_mrp","product_qty","date_finished"]]
ops_def.head()


,name,product_ref,nom_produccion,producto,valor_unitario_mrp,product_qty,date_finished
0,O02445,DAB02995025,DA2995,"DA2995 FILTRO AIRE TOYOTA, MAZDA,FORD",7481.39,1001.0,2018-03-28 13:00:43
1,MO13400,DAB02926025,DA2926,DA2926 FILTRO AIRE DONSSON - INTERNATIONAL 430...,36862.48,500.0,2025-10-31 15:53:18
3,MO13385,DAB08221025,DA8221,DA8221 FILTRO AIRE - FAW EN DESARROLLO,37570.72,30.0,2025-10-31 15:46:26
4,MO13368,DAB12872025,DA2872A,DA2872A FILTRO AIRE- AGRALE,25006.64,200.0,2025-10-31 14:52:53
5,MO13367,DAB02982025,DA2982,DA2982 FILTRO AIRE DONSSON - IHC 7600i MOTOR C...,46399.38,500.0,2025-10-31 10:17:44


In [16]:
import pandas as pd
import numpy as np


ops_def = ops_def.copy()

# Asegúrate que las fechas estén en datetime
ops_def["date_finished"] = pd.to_datetime(ops_def["date_finished"])

# Ordenar por producto y fecha
ops_sorted = ops_def.sort_values(["product_ref", "date_finished"])


ops_sorted['es_ultimo'] = ops_sorted.groupby('product_ref')['date_finished'].transform('max') == ops_sorted['date_finished']

# 2️⃣ Crear las columnas del 1 anterior
ops_sorted["costo_anterior"] = ops_sorted.groupby("product_ref")["valor_unitario_mrp"].shift(1)
ops_sorted["op_anterior"]    = ops_sorted.groupby("product_ref")["name"].shift(1)
ops_sorted["qty_anterior"]   = ops_sorted.groupby("product_ref")["product_qty"].shift(1)
ops_sorted["fecha_anterior"] = ops_sorted.groupby("product_ref")["date_finished"].shift(1)

# 3️⃣ Crear las columnas del 2 anteriores
ops_sorted["costo_anterior_2"] = ops_sorted.groupby("product_ref")["valor_unitario_mrp"].shift(2)
ops_sorted["op_anterior_2"]    = ops_sorted.groupby("product_ref")["name"].shift(2)
ops_sorted["qty_anterior_2"]   = ops_sorted.groupby("product_ref")["product_qty"].shift(2)
ops_sorted["fecha_anterior_2"] = ops_sorted.groupby("product_ref")["date_finished"].shift(2)

# 4️⃣ Variaciones con el 1 anterior (manteniendo los nombres originales)
ops_sorted["variacion"] = ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"]
ops_sorted["variacion_abs"] = np.abs(ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"])
ops_sorted["variacion_pct"] = np.where(
    ops_sorted["costo_anterior"] == 0,
    np.nan,
    (ops_sorted["variacion_abs"] / ops_sorted["costo_anterior"])
)

# 5️⃣ Variaciones con el 2 anterior (con sufijo _2)
ops_sorted["variacion_2"] = ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"]
ops_sorted["variacion_abs_2"] = np.abs(ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"])
ops_sorted["variacion_pct_2"] = np.where(
    ops_sorted["costo_anterior_2"] == 0,
    np.nan,
    (ops_sorted["variacion_abs_2"] / ops_sorted["costo_anterior_2"])
)

# 1. Selecciona las columnas que deseas promediar.
columnas_de_costo = ["valor_unitario_mrp", "costo_anterior_2", "costo_anterior"] # Ajusta los nombres de las 3 columnas

# 2. Calcula el promedio a lo largo del eje de las filas (axis=1).
ops_sorted["promedio"] = ops_sorted[columnas_de_costo].mean(axis=1)


# impacto firmado (puede ser positivo o negativo)
ops_sorted["impacto_signed"] = (ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"]) * ops_sorted["product_qty"]
# impacto firmado (puede ser positivo o negativo)
ops_sorted["impacto_signed_2"] = (ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"]) * ops_sorted["product_qty"]



# impacto absoluto (valor absoluto en pesos, usando qty actual)
ops_sorted["impacto_abs"] = ops_sorted["variacion_abs"] * ops_sorted["product_qty"]
ops_sorted["impacto_abs_2"] = ops_sorted["variacion_abs_2"] * ops_sorted["product_qty"]






def flag_variacion(pct, abs_val):
    if pd.isna(pct):
        return "Sin histórico"
    if pct > 20 or abs_val > 5000000:   # ejemplo: 20% o más de $5MM
        return "Rojo"
    if pct > 5 or abs_val > 1000000:    # ejemplo: entre 5% y 20% o más de $1MM
        return "Amarillo"
    return "Verde"

ops_sorted["flag_var1"] = ops_sorted.apply(lambda r: flag_variacion(r["variacion_pct"], r["impacto_abs"]), axis=1)
ops_sorted["flag_var2"] = ops_sorted.apply(lambda r: flag_variacion(r["variacion_pct_2"], r["impacto_abs_2"]), axis=1)



ops_sorted = ops_sorted.merge(test_def , on ="nom_produccion" ,how="left")



# Exportar a Excel
ops_sorted.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/historial_costos_con_variacion.xlsx", index=False)

print("✅ Archivo generado: historial_costos_con_variacion.xlsx")

#Tarda 20 seg


✅ Archivo generado: historial_costos_con_variacion.xlsx


In [9]:
prototypes = prototypes[["valor_unitario_mrp","date_finished","product_qty","name","product_ref","product_ref","nom_produccion"]]

In [10]:
sin_ventas = ops_sorted[ops_sorted["Clasificacion"].isna()]
sin_ventas= sin_ventas[["valor_unitario_mrp","date_finished","product_qty","name","product_ref","product_ref","nom_produccion"]]

# Suponiendo que el primer DF se llama 'prototypes' 
# y el segundo, el que no tenía clasificación, se llama 'sin_ventas'

df_unido = pd.concat([prototypes, sin_ventas], ignore_index=True)

df_unido["Total"] = df_unido["product_qty"] * df_unido["valor_unitario_mrp"].mean()

df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   valor_unitario_mrp  358 non-null    float64       
 1   date_finished       358 non-null    datetime64[ns]
 2   product_qty         358 non-null    float64       
 3   name                358 non-null    object        
 4   product_ref         358 non-null    object        
 5   product_ref         358 non-null    object        
 6   nom_produccion      358 non-null    object        
 7   Total               358 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 22.5+ KB


In [11]:
df_unido.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/prototipos.xlsx")